In [ ]:
import csv
import sys
import pandas as pd
import os
import glob
import itertools
import numpy as np
from collections import Counter
import matplotlib.pyplot as plt
import math

path = r'/soe/cchakrab/Reduce_WAF_GC/data/'
all_files = glob.glob(os.path.join(path, "synthetic_dataprep_B.csv"))
print(len(all_files))

df_out = pd.DataFrame(columns = ['LBA','Death_time'])


def dataprep(f):
    print("Working with file:" + str(f))
    cols = ['IO_Num','LBA']
    df = pd.read_csv(f,engine='python',skiprows =1,header=None,na_values=['-1'], index_col=False)
    df.columns = cols
    print("Number of IO Accesses: ",len(df))
    
    new_lba_list = df['LBA'].tolist()
    
    # Compute death time per LBA and add a column
    death_time_list = []
    counter = -1

    while(counter < len(new_lba_list) - 1):
        counter = counter + 1
        if(counter >10000 and counter%10000==0):
            print("Counter reached :"+ str(counter) +"   Total IO: " + str(len(new_lba_list)))
            tmp_lba_list = new_lba_list[:len(death_time_list)]
            df_out['LBA'] = tmp_lba_list
            df_out['Death_time'] = death_time_list
            df_out.to_csv(f + "_dataprep_heiner_deathtime_added_incomplete.csv")
            print(f, 'File Written..Still incomplete')
    
    
        lba = new_lba_list[counter]

        if(new_lba_list[(counter+1):].count(lba) == 0):
            death_time_list.append(-1)
        else:
            next_lba_counter = counter + 1 
            next_lba = new_lba_list[next_lba_counter]
            death_counter = 1
            while(next_lba != lba):
                next_lba_counter = next_lba_counter + 1
                next_lba = new_lba_list[next_lba_counter]
                death_counter = death_counter + 1
                if(next_lba_counter == len(new_lba_list) - 1):
                    break
            if(next_lba_counter >= len(new_lba_list) - 2):
                death_time_list.append(-1)
            else:
                death_time_list.append(death_counter)

                

    while(len(death_time_list) !=len(new_lba_list)):
        new_lba_list.pop()

    df_out['LBA'] = new_lba_list
    df_out['Death_time'] = death_time_list
    df_out.to_csv(f + "_dataprep_heiner_deathtime_added_complete.csv")
    print(f, ' ---  Data Prep Done')
    

for f in all_files:
    dataprep(f)